Lesson 5

---

# 🧑‍🍳 LangChain: Memory

## Outline
* Conversation<font color="cyan">Buffer</font>Memory
* Conversation<font color="cyan">BufferWindow</font>Memory
* Conversation<font color="cyan">TokenBuffer</font>Memory
* Conversation<font color="cyan">Summary</font>Memory


> 🔴 <font color="red">Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the class.</font>

## 🧠 Conversational memory 

<h4>Large Language Models are <font color="yellow">stateless</font> = each transaction is independent which means that they don't remember 😱 </h4>

> However, a conversational system should be able to access some window of past messages directly `MEMORY`. A memory system needs to support two basic actions: `reading` and `writing`.

<img src="./img/lc_memory.png" width="800px"/>

Chatbots simulate the presence of memory by incorporating `contextual` information from previous messages or additional details within the conversation. `LangChain` has outlined specific methods to maintain this context through a set of memory classes that replicate various memory behaviors and they are the following:

* Conversation<font color="cyan">Buffer</font>Memory
* Conversation<font color="cyan">BufferWindow</font>Memory
* Conversation<font color="cyan">TokenBuffer</font>Memory
* Conversation<font color="cyan">Summary</font>Memory
* Conversation<font color="cyan">KnowledgeGraph</font>Memory (Conversation<font color="cyan">KG</font>Memory)
* Conversation<font color="cyan">Entity</font>Memory
* Vector Store


Conversational memory is what enables chatbots to respond to our queries in a conversational manner. It allows for coherent conversations by considering past interactions, rather than treating each query as independent. Without conversational memory, chatbots would lack the ability to remember and build upon previous interactions.

👉 Explore additional details about LangChain Memories at  https://python.langchain.com/docs/modules/memory/

## ⚙️ Setup

In [1]:
# Open AI
from openai import OpenAI

# LangChain
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# Local
from util import local_settings
from env_colors import TerminalTextColor

# Settings
model="gpt-3.5-turbo"

print("First LLM API example")
print(f"✅ OpenAI Key loaded (...{local_settings.OPENAI_API_KEY[20:-20]}...)")
print(f"✅ Model: {model}")

import warnings
warnings.filterwarnings('ignore')

First LLM API example
✅ OpenAI Key loaded (...UQiT3BlbkFJ...)
✅ Model: gpt-3.5-turbo


## Conversation<b><font color="cyan">Buffer</font></b>Memory

This memory allows for storing messages and then extracts the messages in a variable.

In [2]:
llm = ChatOpenAI(temperature=0.0, model=model)

memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

In [3]:
conversation.predict(input="Hi, my name is Fernando")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Fernando
AI:

> Finished chain.


"Hello Fernando! It's nice to meet you. How can I assist you today?"

In [4]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Fernando
AI: Hello Fernando! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 is equal to 2.'

In [5]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Fernando
AI: Hello Fernando! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is Fernando.'

In [6]:
print(memory.buffer)

Human: Hi, my name is Fernando
AI: Hello Fernando! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI: Your name is Fernando.


In [7]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Fernando\nAI: Hello Fernando! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1+1 is equal to 2.\nHuman: What is my name?\nAI: Your name is Fernando."}

In [8]:
memory = ConversationBufferMemory()

In [9]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})

In [10]:
print(memory.buffer)

Human: Hi
AI: What's up


In [11]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [12]:
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [13]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

## Conversation<b><font color="cyan">BufferWindow</font></b>Memory

Conversation<font color="cyan">BufferWindow</font>Memory keeps a list of the interactions of the conversation over time. It only uses the last `K` interactions. This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [14]:
from langchain.memory import ConversationBufferWindowMemory

In [15]:
memory = ConversationBufferWindowMemory(k=1)

In [16]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [17]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [18]:
llm = ChatOpenAI(temperature=0.0, model=model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False
)

In [19]:
conversation.predict(input="Hi, my name is Fernando")

"Hello Fernando! It's nice to meet you. How can I assist you today?"

In [20]:
conversation.predict(input="What is 1+1?")

'1+1 is equal to 2.'

In [21]:
conversation.predict(input="What is my name?")

"I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation."

## Conversation<b><font color="cyan">TokenBuffer</font></b>Memory

Conversation<font color="cyan">TokenBuffer</font>Memory keeps a buffer of recent interactions in memory, and uses `token length` rather than `number of interactions`(`k`) to determine when to flush interactions.

In [22]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0, model=model)

In [23]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"},
                    {"output": "Charming!"})

In [24]:
memory.load_memory_variables({})

{'history': 'AI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## Conversation<b><font color="cyan">Summary</font></b>Memory

In [25]:
from langchain.memory import ConversationSummaryBufferMemory

In [26]:
# create a long string
schedule = """There is a meeting at 8am with your product team.
You will need your PowerPoint presentation prepared.
9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool.
At Noon, lunch at the Italian restaurant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo."""

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)

memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"},
                    {"output": f"{schedule}"})

In [27]:
memory.load_memory_variables({})

{'history': 'System: The human greets the AI and asks what is on the schedule for the day.\nAI: There is a meeting at 8am with your product team.\nYou will need your PowerPoint presentation prepared.\n9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool.\nAt Noon, lunch at the Italian restaurant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.'}

In [28]:
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

In [29]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human greets the AI and asks what is on the schedule for the day.
AI: There is a meeting at 8am with your product team.
You will need your PowerPoint presentation prepared.
9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool.
At Noon, lunch at the Italian restaurant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.
Human: What would be a good demo to show?
AI:

> Finished chain.


'A good demo to show would be the Language Model API. It showcases the capabilities of our AI technology by generating coherent and contextually relevant text based on user input. It can be used for a variety of applications such as chatbots, content generation, and language translation.'

In [30]:
memory.load_memory_variables({})

{'history': 'System: The human greets the AI and asks what is on the schedule for the day. The AI informs the human that there is a meeting at 8am with the product team and advises them to have their PowerPoint presentation prepared. The AI also mentions that from 9am to 12pm, the human has time to work on their LangChain project, which will be quick due to the power of LangChain. At noon, the human has a lunch appointment at an Italian restaurant with a customer who is driving from over an hour away to learn about the latest in AI. The AI reminds the human to bring their laptop to show the latest LLM demo.\nHuman: What would be a good demo to show?\nAI: A good demo to show would be the Language Model API. It showcases the capabilities of our AI technology by generating coherent and contextually relevant text based on user input. It can be used for a variety of applications such as chatbots, content generation, and language translation.'}

##  Conversation<font color="cyan"><b>Entity</b></font>Memory

In [31]:
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from langchain.memory import ConversationEntityMemory

from pydantic import BaseModel
from typing import List, Dict, Any

In [33]:
llm = ChatOpenAI(temperature=0.0, model=model)

memory = ConversationEntityMemory(llm=llm)
_input = {"input": "Deven & Sam are working on a hackathon project"}
memory.load_memory_variables(_input)
memory.save_context(
    _input,
    {"output": " That sounds like a great project! What kind of project are they working on?"}
)

In [34]:
memory.load_memory_variables({"input": 'who is Sam'})

{'history': 'Human: Deven & Sam are working on a hackathon project\nAI:  That sounds like a great project! What kind of project are they working on?',
 'entities': {'Sam': 'Sam is currently working on a hackathon project with Deven.'}}

In [41]:
conversation = ConversationChain(
    llm=llm,
    verbose=False,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,

    memory=memory
)

In [42]:
conversation.predict(input="Deven & Sam are working on a hackathon project")

'That sounds like a great project! What kind of project are Deven and Sam working on for the hackathon?'

In [43]:
conversation.memory.entity_store.store

{'Deven': 'Deven is currently working on a hackathon project with Sam.',
 'Sam': 'Sam is currently working on a hackathon project with Deven.'}

In [44]:
conversation.predict(input="They are trying to add more complex memory structures to Langchain")

"That's interesting! Adding more complex memory structures to Langchain sounds like a challenging and exciting project for the hackathon. Can you provide more details about what kind of memory structures they are trying to incorporate?"

In [45]:
conversation.predict(input="What do you know about Deven & Sam?")

"Deven & Sam are the individuals working on the hackathon project. Unfortunately, I don't have any specific information about them beyond what you've mentioned. Is there anything specific you would like to know about them?"

In [46]:
conversation.predict(input="They are adding in a key-value store for entities mentioned so far in the conversation.")

"That's a great addition! Adding a key-value store for entities mentioned in the conversation can be very useful for organizing and retrieving information. It allows for efficient storage and retrieval of data based on a unique key associated with each entity. This can help in maintaining a structured memory system within Langchain and enable easy access to previously mentioned entities. Is there anything else you would like to know or discuss about this project?"

In [47]:
conversation.predict(input="What do you know about Deven & Sam?")

"Deven & Sam are the individuals working on the hackathon project. Unfortunately, I don't have any specific information about them beyond what you've mentioned. Is there anything specific you would like to know about them?"

In [50]:
from pprint import pprint
pprint(conversation.memory.entity_store.store)

{'Deven': 'Deven is currently working on a hackathon project with Sam.',
 'Deven & Sam': 'Deven & Sam are the individuals working on the hackathon '
                'project.',
 'Langchain': 'Langchain is a project that Deven and Sam are working on for '
              'the hackathon, where they are trying to add more complex memory '
              'structures.',
 'Sam': 'Sam is currently working on a hackathon project with Deven.'}


<h3><font color="Yellow">👋 the end </font></h3>